### Liempieza

In [ ]:
import numpy as np
import pandas as pd
import re

from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import csr_matrix

In [ ]:
PATH_tweets = '../Data/tweets.csv'

In [ ]:
tweets = pd.read_csv(
    PATH_tweets,
    usecols=['id', 'screen_name', 'text'], 
    index_col='id', 
    dtype={'screen_name': str, 'text': str}
    )
tweets = tweets.drop_duplicates(subset='text', keep=False)
# tweets = tweets.drop_duplicates(subset=['screen_name', 'text'], keep=False)
tweets = tweets.dropna()

Cargamos csv, botamos filas identicas y nulas.

In [ ]:
def limpiar_retweet(tweet:str):
    if tweet.startswith('RT'):
        tweet = ''.join(tweet.split(': ')[1:])
    return tweet

def limpiar_hashtag(tweet:str):
    tweet = re.sub(r'#\w+', '', tweet)
    return tweet

def limpiar_url(tweet:str):
    tweet = re.sub(r'http\S+', '', tweet)
    return tweet

def limpiar_emoji(tweet:str):
    tweet = re.sub(r'\\x\w+', '', tweet)
    return tweet

def limpiar_puntuacion(tweet:str):
    tweet = re.sub(r'[^\w\s@]', '', tweet)
    return tweet

def limpiar_espacios(tweet:str):
    tweet = re.sub(r'\s+', ' ', tweet)
    return tweet

def limpiar_mayusculas(tweet:str):
    tweet = tweet.lower()
    return tweet

def remplazar_tildes(tweet:str):
    tweet = tweet.replace('á', 'a')
    tweet = tweet.replace('é', 'e')
    tweet = tweet.replace('í', 'i')
    tweet = tweet.replace('ó', 'o')
    tweet = tweet.replace('ú', 'u')
    return tweet

def limpiar_texto(tweet:str):
    tweet = limpiar_retweet(tweet)
    tweet = limpiar_hashtag(tweet)
    tweet = limpiar_url(tweet)
    tweet = limpiar_emoji(tweet)
    tweet = limpiar_puntuacion(tweet)
    tweet = limpiar_espacios(tweet)
    tweet = limpiar_mayusculas(tweet)
    tweet = remplazar_tildes(tweet)
    tweet = tweet.strip()
    return tweet

tweets['text'] = tweets.loc[:, 'text'].apply(limpiar_texto)

Normalizamos los tweets, limpiamos los emojis, sacamos los tildes, las mayusculas, los puntos, los urls, retweets y hashtags.

### Hashing

In [ ]:
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm

In [167]:
def shingles(k, words: np.ndarray) -> list:
    temp = []
    for i in range(0, len(words)):
        temp.append(' '.join(words[i:i + k]))
    return list(set(temp))

Creamos los shingles.

In [ ]:
def caracteristica(k:int=2, sample_size:int=None, df:pd.DataFrame=tweets) -> csr_matrix:
    if sample_size:
        df = df.sample(sample_size)
    df['shingles'] = df['text'].str.split().apply(lambda x: np.array(x)).apply(lambda x: shingles(k, x))
    shings = df['shingles'].to_numpy()
    mlb = MultiLabelBinarizer(sparse_output=True)
    caracteristica = mlb.fit_transform(shings)

    return caracteristica

Creamos una matriz caracteristica en formato sparse de los tweets y shingles, si un shingle esta en un tweet, entonces marcamos ese registro con un 1, si no, con un 0. Esta en formato sparse ya que no tenemos 30 terabytes para poder crear la matriz caracteristica completa.

In [ ]:
def minhash_vector(sparse_vector:csr_matrix, num_perm:int=128):
    minhash = MinHash(num_perm=num_perm)
    for index in sparse_vector.indices:
        minhash.update(str(index).encode('utf8'))
    return minhash

In [ ]:
k = 2
s = 0.2
threshold = 1 - s
n_perm = 128

In [ ]:
matrix = caracteristica(k)

In [ ]:
lsh = MinHashLSH(threshold=threshold, num_perm=n_perm)
hashes = []

for i in tqdm(range(matrix.shape[0])):
    hashes.append((i, minhash_vector(matrix[i], num_perm=n_perm)))

with lsh.insertion_session() as session:
    for i, minhash in tqdm(hashes):
        session.insert(i, minhash)

Usamos los metodos Minhash y MinHashLSH de la libreria datasketch para hashear la matriz caracteristica sparse, ocupamos 128 hashes y la muestra completa deberia hashearse en 20 min aprox. El metodo MinHashLSH ya tine implementado un sistema de querys y el threshold representa la similaridad de jaccard minima que pueden tener dos tweets para ser similares.

### Querys

Hacemos una query para un tweet y persona en especifico, en donde se retorna los tweets similares a este tweet mencionado anteriormente. Los parametros para esto son k = 2 y s = 0.2, tambien creamos un diccionario en donde contamos cuantas veces aparece cada usuario, de esta manera podemos ver que usuario comparte una mayor cantidad de tweets similares con la persona original.

In [ ]:
indice = 1007

m = minhash_vector(matrix[indice])
aprox = lsh.query(m)

original = tweets.iloc[indice]
print(f'{original["screen_name"] + ";".ljust(10)} {original["text"]}\n')

repeticiones = {}
for i in aprox:
    twt = tweets.iloc[i]
    print( f'{(str(i) + ",").ljust(8)} {twt["screen_name"].strip().ljust(20)}:'.ljust(30), twt['text'] )
    repeticiones[twt.screen_name] = repeticiones.get(twt.screen_name, 0) + 1

Seleccionamos el usuario que tiene mas tweets similares con el usuario y tweet original de la query, y encontramos todos los tweets de estas dos personas.

In [ ]:
mas_repetido = max(repeticiones)
tweets = pd.read_parquet('../Data/tweets.parquet')
par_con_nombres = []
par_sin_nombres = []

for c, name in enumerate(tweets['screen_name']):
    if name == original['screen_name']:
        tweet = tweets['text'].iloc[c]
        par_con_nombres.append((name, tweet))
        par_sin_nombres.append(tweet)
    elif name == mas_repetido:
        tweet = tweets['text'].iloc[c]
        par_con_nombres.append((name, tweet))
        par_sin_nombres.append(tweet)

Calculamos shingles de los tweets hechos por estas dos personas.

In [ ]:
def shingles(k, words: np.ndarray):
    shingles = []
    for i in range(0, len(words)):
        shing = ' '.join(words[i:i+k])
        if len(shing.split()) == k:
            shingles.append(shing)
    return list(set(shingles))


shings_list = []
for twt in par_sin_nombres:
    twt_shings = shingles(2, np.array(twt.split()))
    if twt_shings != []:
        for shing in twt_shings:
            shings_list.append(shing)

Creamos matriz caracteristica de los shingles y tweets, asignamos un 1 si el shingle se encuentra en el tweet, y un 0 si no.

In [184]:
def create_characteristic_matrix(list1, list2):
    matrix = []
    for string1 in list1:
        row = []
        for string2 in list2:
            if string1 in string2:
                row.append(1)
            else:
                row.append(0)
        matrix.append(row)
    return matrix

car = create_characteristic_matrix(shings_list, par_sin_nombres)

Calculamos la similitud de jaccard en la matriz caracteristica, y retorna las posiciones de los tweets similares si la similitud de jaccard entre cada tweet es mayor o igual a un parametro 's'.

In [198]:
def calculate_jaccard_difference(matrix, s):
    num_cols = len(matrix[0])
    jaccard_matrix = []

    for i in range(num_cols):
        for j in range(i + 1, num_cols):
            intersection = np.sum(np.logical_and(matrix[:, i], matrix[:, j]))
            union = np.sum(np.logical_or(matrix[:, i], matrix[:, j]))

            if union != 0:
                jaccard = intersection / union
                if jaccard >= s:
                    jaccard_matrix.append([i, j, jaccard])

    return jaccard_matrix

posiciones = calculate_jaccard_difference(np.array(car), 0.3)
len(posiciones)

16

Finalmente printeamos el par de tweets similares entre las dos personas.

In [200]:
for x in posiciones:
    if par_con_nombres[x[0]][0] != par_con_nombres[x[1]][0]:
        # print(par_con_nombres[x[0]], par_con_nombres[x[1]])
        par_1 = par_con_nombres[x[0]] ; par_2 = par_con_nombres[x[1]]
        print(
            f'{par_1[0]}: {par_1[1].strip()}. \n{par_2[0] + ": " + par_2[1]}. \n'
        )



IanDiazc: la derecha quiere permanecer en la casa del golpista y el cuatro ojos. 
z4ncudoelectric: cero esperanza en la derecha tal cual. 

z4ncudoelectric: con esto queda toda la derecha inhabilitada. 
IanDiazc: con eso quedan fuera la mitad de la derecha manilarga y raspador a de ollas bien. 

z4ncudoelectric: con esto queda toda la derecha inhabilitada. 
IanDiazc: llamen al sr lapiz para que le explique a la derecha del retrazo. 

